In [1]:
using CSV
using Dates
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-02-27_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,target_date,hospitalizations
,Int64,String,String?,String,Int64,Float64?,Float64?,String,Float64?
1,10,PRATTVILLE BAPTIST HOSPITAL,10108,1/29/21,1001,1.0,0.189189,2/3/21,3.40048
2,11,SOUTH BALDWIN REGIONAL MEDICAL CENTER,10083,1/29/21,1003,0.323413,0.406061,2/3/21,4.03381
3,12,NORTH BALDWIN INFIRMARY,10129,1/29/21,1003,0.162858,0.348485,2/3/21,2.03127
4,13,THOMAS HOSPITAL,10100,1/29/21,1003,0.513729,0.197026,2/3/21,6.40756
5,14,MEDICAL CENTER BARBOUR,10069,1/29/21,1005,1.0,0.227273,2/3/21,1.59001
6,15,BIBB MEDICAL CENTER,10058,1/29/21,1007,0.814815,0.0,2/3/21,0.767816
7,16,ST VINCENTS BLOUNT,11305,1/29/21,1009,0.962963,0.0526316,2/3/21,2.62851
8,17,BULLOCK COUNTY HOSPITAL,10110,1/29/21,1011,0.0,missing,2/3/21,0.0
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,10150,1/29/21,1013,0.962963,0.555556,2/3/21,0.840803


In [4]:
function firstval(xs)
    isbad(x) = isnothing(x) || ismissing(x)
    if all(isbad.(xs))
        return missing
    else
        return xs[findfirst(x -> !isbad(x), xs)]
    end
end;

In [5]:
ccn_lookup_df = combine(groupby(rawdata, [:Column1, :hospital_name]), :ccn => firstval => :ccn)
ccn_lookup = Dict(row.Column1 => row.ccn for row in eachrow(ccn_lookup_df));

In [6]:
data = select(rawdata,
    :hospital_name => :hospital,
    :Column1 => ByRow(x -> ccn_lookup[x]) => :hospital_id,
    :target_date => ByRow(d -> Date(d, dateformat"m/d/y")+Year(2000)) => :date,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalizations" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-03,0.0,1.29152,1.29152
2,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-04,0.0,1.32475,1.32475
3,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-05,0.0,1.27687,1.27687
4,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-06,0.0,1.26223,1.26223
5,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-07,0.0,1.28687,1.28687
6,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-08,0.0,1.25579,1.25579
7,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-09,0.0,1.23703,1.23703
8,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-10,0.0,1.19365,1.19365
9,ABBEVILLE AREA MEDICAL CENTER,421301,2021-02-11,0.0,1.22531,1.22531


In [7]:
data |> CSV.write("../data/hhs_forecast_2021_02_08.csv")

"../data/hhs_forecast_2021_02_08.csv"